<a href="https://colab.research.google.com/github/garthajon/QuantFinanceIntro/blob/main/zscore_stock_screener_weekly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


# ---------------- SECTOR GROUPS ----------------
sectors = {
    "Technology": [
        "AAPL","MSFT","NVDA","GOOGL","GOOG","AMZN","META","AVGO","AMD","ADBE","CRM","INTC","ORCL","CSCO","TXN","QCOM",
        "NOW","AMAT","MU","PANW","SNPS","INTU","LRCX","ADI","NXPI","CDNS","MCHP","KLAC","CRWD","FTNT","MSI","APH",
        "ANET","PAYC","DDOG","MDB","ZS","TEAM","WDAY","SHOP","NET","OKTA","HPQ","DELL","TTD","UBER","ABNB",
        "COIN","PATH","SNOW","PLTR","SMCI","ENPH","SEDG","ON","MPWR","MRVL","ASML","TER","KEYS","HUBS","AKAM",
        "CTSH","CDW","VRSN"
    ],
    "Energy": [
        "XOM","CVX","COP","SLB","EOG","PSX","MPC","OXY","BKR","VLO","HAL","KMI","WMB","DVN","APA","FANG",
        "CTRA","OKE","CVE","CNQ","SU","ENB","EQT","RRC","OVV","AR","TRGP","MTDR","SM","MUR","PBF",
        "NRG","DTE","NI","CMS","CNP","AES","ETR","PNW","EIX","AWK","WEC","ATO"
    ],
    "Financials": [
        "JPM","GS","MS","BLK","BRK-B","V","MA","PYPL","BAC","C","WFC","USB","PNC","TFC","BK","NTRS","STT","SCHW",
        "CME","ICE","SPGI","MCO","AFL","MET","PRU","AIG","CB","PGR","ALL","TRV","FITB","KEY","RF","CFG","ZION",
        "HBAN","ALLY","MTB","COF","AMP","BEN","TROW","IVZ","NDAQ","RJF","AJG","MMC","HIG","CINF",
        "BRO","CBOE","MKTX"
    ],
    "Pharma": [
        "PFE","LLY","ABBV","MRK","BMY","GILD","VRTX","REGN","AMGN","BIIB","ZTS","MRNA","IDXX","ALNY","INCY","TECH",
        "HCM","UTHR","IONS","NBIX","RPRX"
    ],
    "Healthcare": [
        "UNH","JNJ","TMO","DHR","ISRG","ZBH","EW","MDT","BSX","SYK","HUM","CNC","CI","HCA","DGX","LH","BIO","ILMN",
        "ABT","DXCM","RMD","STE","HOLX","A","MTD","WAT","BAX"
    ],
    "Industrials": [
        "CAT","DE","HON","GE","LMT","BA","NOC","RTX","GD","MMM","EMR","ETN","ITW","PH","ROK","CMI","IR","JCI","AOS",
        "AME","ALLE","HWM","LHX","OTIS","TT","TDG","XYL","CSX","UNP","NSC","FDX","UPS","UAL","DAL","LUV","AAL",
        "FAST","URI","PCAR","TXT","PWR","EME","SNA"
    ],
    "Consumer": [
        "HD","LOW","COST","WMT","TGT","NKE","DIS","NFLX","TSLA","SBUX","KO","PEP","PG","KMB","CL","EL","GIS","MDLZ",
        "K","HSY","PM","MO","TAP","CLX","HRL","KR","WBA","DG","DLTR","ROST","TJX","BBY","MCD","YUM","CMG","DPZ",
        "DRI","BKNG","EXPE","LYV","RCL","CCL","MAR","HLT","H","WYNN","MGM"
    ],
    "Utilities & Real Estate": [
        "D","DUK","NEE","SO","AEP","EXC","PEG","ED","XEL","PCG","EIX","ES","FE","CMS","AEE","AWK","WEC","ATO",
        "PLD","AMT","EQIX","CCI","PSA","SPG","O","WELL","VTR","ARE","IRM","EXR"
    ],
    "Misc": [
        "PARA","WBD","FOX","FOXA","NWS","NWSA","CEG","CPRT","F","GM","STLA","BBWI","CHTR","LKQ","AAP","AZO",
        "ORLY","TSCO","GPC","LEG","HAS","MAT","POOL","LEN","DHI","PHM","TOL","NVR","RHI","ROL","RSG","WM","GWW",
        "PAYX","ETSY","EBAY","DUOL"
    ]
}

# ---------------- FETCH DATA ----------------
all_tickers = sorted(list({t for s in sectors.values() for t in s}))
end = datetime.today()
start = end - timedelta(weeks=52)

print(f"Downloading {len(all_tickers)} tickers (1-year weekly)...\n")
data = yf.download(all_tickers, start=start, end=end, interval="1wk")["Close"]
data = data.dropna(axis=1, how="any")

# ---------------- COMPUTE Z-SCORES ----------------
returns = data.pct_change().dropna()
zscores = (returns - returns.mean()) / returns.std()
latest_z = zscores.iloc[-1]

# ---------------- DISPLAY EXTREMES BY SECTOR ----------------
pd.set_option("display.width", 160)
pd.set_option("display.max_rows", 100)

for sector, tickers in sectors.items():
    tickers_in_data = [t for t in tickers if t in latest_z.index]
    if not tickers_in_data:
        continue

    zvals = latest_z[tickers_in_data].dropna()
    high = zvals[zvals > 2].sort_values(ascending=False)
    low = zvals[zvals < -2].sort_values()

    if len(high) == 0 and len(low) == 0:
        continue

    print(f"\n=== {sector.upper()} ===")
    if not high.empty:
        print("\nAbove +2σ:")
        print(high.to_frame("Z-Score"))
    if not low.empty:
        print("\nBelow -2σ:")
        print(low.to_frame("Z-Score"))


/tmp/ipython-input-229537058.py:63: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(all_tickers, start=start, end=end, interval="1wk")["Close"]
[*********************100%***********************]  360 of 360 completed



=== TECHNOLOGY ===

Above +2σ:
         Z-Score
Ticker          
MU      2.041176

=== PHARMA ===

Above +2σ:
         Z-Score
Ticker          
PFE     3.788303
TECH    3.704275
BIIB    3.540986
MRK     3.137356
AMGN    2.462209
LLY     2.413199

=== HEALTHCARE ===

Above +2σ:
         Z-Score
Ticker          
TMO     3.621142
DHR     3.525156
A       3.264288
CI      2.354132
WAT     2.285695
MTD     2.090357
BIO     2.073241

Below -2σ:
         Z-Score
Ticker          
DGX    -2.283674

=== CONSUMER ===

Above +2σ:
         Z-Score
Ticker          
K       5.507217

=== UTILITIES & REAL ESTATE ===

Above +2σ:
         Z-Score
Ticker          
AEP     2.081863

=== MISC ===

Above +2σ:
         Z-Score
Ticker          
STLA    2.420706

Below -2σ:
         Z-Score
Ticker          
NWSA   -2.221334
